In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras import regularizers

In [23]:
!pip install xgboost tensorflow keras tensorflow._api

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/ed/1a/b4ab4b8f8b3a41fade4899fd00b5b2d2dad0981f3e1bb10df4c522975fd7/tensorflow-2.15.0.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached tensorflow-2.15.0.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
ERROR: Could not find a version that satisfies the requirement tensorflow._api (from versions: none)
ERROR: No matching distribution found for tensorflow._api


In [ ]:
df = pd.read_csv('StudentsPerformance.csv')

In [ ]:
df.head()

In [ ]:
num_features = ['math score', 'reading score', 'writing score']
cat_features = ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course']

In [ ]:

df[num_features].hist(bins=30, figsize=(10, 8))
for feature in cat_features:
    print(df[feature].value_counts())

In [ ]:
corr_matrix = df.corr()
print(corr_matrix['math score'].sort_values(ascending=False))


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Linear Regression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
lin_reg_score = lin_reg.score(X_test, y_test)
print('Linear Regression Test score:', lin_reg_score)

In [ ]:
# Decision Tree
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
tree_score = tree.score(X_test, y_test)
print('Decision Tree Test score:', tree_score)

In [ ]:
# Random Forest
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
rf_score = rf.score(X_test, y_test)
print('Random Forest Test score:', rf_score)

In [ ]:
# XGBoost
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
xgb_score = xgb.score(X_test, y_test)
print('XGBoost Test score:', xgb_score)

In [ ]:
# 3. Xây dựng Neural Network
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))  # Dropout layer to prevent overfitting
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
# Fit the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

In [ ]:
# 4. Đánh giá mô hình Neural Network trên tập test
nn_score = model.evaluate(X_test, y_test)
print('Neural Network Test Loss:', nn_score)

In [ ]:
# 5. Phân tích các trường hợp sai và cải tiến
y_pred_nn = model.predict(X_test)
mse_nn = mean_squared_error(y_test, y_pred_nn)
print('Neural Network Mean Squared Error:', mse_nn)

In [ ]:
# Cải tiến mô hình Neural Network
model_reg = Sequential()
model_reg.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_dim=X_train.shape[1]))
model_reg.add(Dropout(0.2))
model_reg.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model_reg.add(Dense(1))

model_reg.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Early stopping and regularization
history_reg = model_reg.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

In [ ]:
# Đánh giá mô hình Neural Network cải tiến trên tập test
nn_reg_score = model_reg.evaluate(X_test, y_test)
print('Regularized Neural Network Test Loss:', nn_reg_score)